In [1]:
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "OECD_file/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = pd.read_csv(file_name, delimiter="|") # load the data
        print(df)

          appln_id        app_nbr  filing  tech_field  many_field  \
0         16427085  EP19780100001    1978        16.0           1   
1         16427086  EP19780100002    1978        31.0           0   
2         16427087  EP19780100003    1978        32.0           1   
3         16427088  EP19780100004    1978        14.0           1   
4         16427089  EP19780100005    1978         8.0           0   
...            ...            ...     ...         ...         ...   
3507894  506385435  EP20190405001    2019         8.0           1   
3507895  506385437  EP20190405002    2019        24.0           0   
3507896  507537560  EP20190460001    2019        35.0           0   
3507897  505638106  EP20190461501    2019         1.0           1   
3507898  506424754  EP20190700348    2019         6.0           0   

         patent_scope  family_size  grant_lag  bwd_cits  npl_cits  ...  \
0                   3            7     1203.0         4         0  ...   
1                   1  

In [2]:
# Now try to pull the patent data combining 2 files to accumulate 10 years
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "Patent_EP/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

patent_data_combine = None

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        patent_data = pd.read_csv(file_name) # load the data
        # print(patent_data)
        if patent_data_combine is None:
            patent_data_combine = patent_data
        else:
            patent_data_combine = patent_data_combine.append(patent_data)

In [3]:
patent_data_combine.sort_values('filing_date')

,publication_number,application_number,text,filing_date
161697,EP-2393505-A1,EP-10705790-A,The present invention relates to the use of a ...,20100102
11332,EP-2376972-A1,EP-10708886-A,"A parallel-sighted, image-reversing prism syst...",20100102
52710,EP-2384189-A2,EP-10726819-A,Improved compositions for tissue augmentation ...,20100103
74016,EP-2374013-A2,EP-10708389-A,Apparatus for radiation mapping of a region co...,20100103
157739,EP-2374128-A2,EP-10703964-A,A device for recording at least part of a data...,20100103
...,...,...,...,...
16172,EP-3572775-A2,EP-19193474-A,A switch control circuit and method for an ele...,20190823
19388,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
19389,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
12887,EP-3564854-A1,EP-19196510-A,Embodiments of the present invention provide a...,20190910


In [4]:
patent_data_combine['new_appl_nbr'] = 'EP20' + patent_data_combine['application_number'].str[3:5] + '0' + patent_data_combine['application_number'].str[5:-2]

In [5]:
patent_data_combine

,publication_number,application_number,text,filing_date,new_appl_nbr
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995
...,...,...,...,...,...
19963,EP-3517373-A1,EP-19152079-A,A seatbelt pretensioning retractor assembly fo...,20190116,EP20190152079
19964,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131
19965,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131
19966,EP-3545862-A2,EP-19165960-A,A method of compressing tissue during a surgic...,20190328,EP20190165960


In [6]:
patent_data_combine.drop_duplicates(subset='new_appl_nbr', keep = 'first', inplace = True)
patent_data_combine

,publication_number,application_number,text,filing_date,new_appl_nbr
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995
...,...,...,...,...,...
19957,EP-3567539-A1,EP-19171027-A,Distributing resources in a predetermined geog...,20190425,EP20190171027
19960,EP-3527317-A1,EP-19156479-A,Embodiments of hybrid engine welding systems a...,20190211,EP20190156479
19962,EP-3517373-A1,EP-19152079-A,A seatbelt pretensioning retractor assembly fo...,20190116,EP20190152079
19964,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131


In [7]:
df_merge = pd.merge(patent_data_combine, df, left_on='new_appl_nbr', right_on='app_nbr', how='inner')

In [8]:
df_merge

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,fwd_cits7,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469,328773105,EP20100773469,2010,25.0,0,...,0,0,NaN,NaN,NaN,0.602041,0.000000,4.0,0.396296,0.393637
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481,333779641,EP20100826481,2010,4.0,1,...,0,0,NaN,NaN,NaN,0.839506,0.111111,NaN,NaN,NaN
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430,341900782,EP20100856430,2010,32.0,0,...,0,0,NaN,NaN,NaN,0.867139,0.224490,NaN,NaN,NaN
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626,339215521,EP20110190626,2010,25.0,0,...,9,4,NaN,NaN,0.180000,0.764543,0.157895,NaN,NaN,NaN
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995,419197262,EP20140172995,2010,24.0,0,...,6,2,NaN,NaN,0.104938,0.887755,0.571429,9.0,0.420142,0.298506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130300,EP-3512074-A1,EP-19150413-A,The invention relates to a permanent magnet ro...,20190104,EP20190150413,505250492,EP20190150413,2019,1.0,0,...,0,0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
1130301,EP-3514984-A1,EP-19152912-A,The invention provides a method for determinin...,20190121,EP20190152912,506365616,EP20190152912,2019,3.0,0,...,0,0,NaN,NaN,NaN,0.908163,0.857143,NaN,NaN,NaN
1130302,EP-3514738-A1,EP-19152968-A,A demand prediction apparatus configured to pr...,20190122,EP20190152968,506385155,EP20190152968,2019,7.0,0,...,0,0,NaN,NaN,NaN,0.000000,1.000000,NaN,NaN,NaN
1130303,EP-3514451-A1,EP-19151645-A,The present invention relates generally to a f...,20190114,EP20190151645,505895332,EP20190151645,2019,1.0,0,...,0,0,NaN,NaN,NaN,0.843750,0.375000,NaN,NaN,NaN


In [9]:
df_merge_quality = df_merge[(df_merge['quality_index_6'] >=0) & (len(df_merge['text']) >0) & (df_merge['filing_date'] < 20160101)]
df_merge_quality

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,fwd_cits7,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469,328773105,EP20100773469,2010,25.0,0,...,0,0,NaN,NaN,NaN,0.602041,0.000000,4.0,0.396296,0.393637
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995,419197262,EP20140172995,2010,24.0,0,...,6,2,NaN,NaN,0.104938,0.887755,0.571429,9.0,0.420142,0.298506
5,EP-2255683-B1,EP-10163204-A,The device has an application device (47) for ...,20100519,EP20100163204,315607513,EP20100163204,2010,34.0,0,...,1,1,NaN,NaN,NaN,0.642361,0.416667,9.0,0.278788,0.367996
6,EP-2262981-B1,EP-10711585-A,The invention relates to a bimetallic valve fo...,20100329,EP20100711585,298884917,EP20100711585,2010,27.0,0,...,0,0,NaN,NaN,NaN,0.842951,0.675676,7.0,0.300938,0.450909
7,EP-2462850-B1,EP-10194739-A,The present invention concerns a beverage prep...,20101213,EP20100194739,329540694,EP20100194739,2010,33.0,0,...,1,1,NaN,NaN,0.000000,0.668639,0.461538,9.0,0.273246,0.367701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843915,EP-2952147-A1,EP-15170632-A,"In some embodiments, a debridement tool (124) ...",20150604,EP20150170632,440974359,EP20150170632,2015,13.0,0,...,0,0,NaN,NaN,NaN,0.894174,0.758621,3.0,0.236467,0.302581
843916,EP-3040395-A1,EP-15198385-A,The present invention discloses an novel organ...,20151208,EP20150198385,447194132,EP20150198385,2015,14.0,1,...,0,0,NaN,NaN,NaN,0.838542,0.125000,3.0,0.188095,0.253368
843918,EP-3121161-A1,EP-15178054-A,The invention relates to an improved method fo...,20150723,EP20150178054,442595431,EP20150178054,2015,14.0,0,...,0,0,NaN,NaN,NaN,0.884898,0.514286,3.0,0.184524,0.210907
843919,EP-3159891-A1,EP-15190987-A,An active road-noise control system and method...,20151022,EP20150190987,445551961,EP20150190987,2015,34.0,0,...,0,0,NaN,NaN,NaN,0.833792,0.636364,3.0,0.270879,0.310780


In [10]:
#
#conditions = [
#    (df_merge_quality['quality_index_6'] >= 0) & (df_merge_quality['quality_index_6'] < 0.15),
#    (df_merge_quality['quality_index_6'] >= 0.15) & (df_merge_quality['quality_index_6'] < 0.3),
#    (df_merge_quality['quality_index_6'] >= 0.3) & (df_merge_quality['quality_index_6'] < 0.45),
#    (df_merge_quality['quality_index_6'] >= 0.45) & (df_merge_quality['quality_index_6'] < 0.6),
#    (df_merge_quality['quality_index_6'] >= 0.6)
#]

#choices = ['0', '1', '2', '3', '4']
#**//

In [11]:
conditions = [

    (df_merge_quality['breakthrough'] != 1),
    (df_merge_quality['breakthrough'] == 1)
]

choices = ['0', '1']

In [12]:
import numpy as np
df_merge_quality['breakthrough_flg'] = np.select(condlist=conditions, choicelist=choices)

/home/nobu_yamaguchi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
def tech_field_cat(subcat):
    if subcat <= 8:
        return '0' # for 1. Electrical engineering
    elif subcat <= 13:
        return '1' # for 2. Instruments
    elif subcat <= 24:
        return '2' # for 3. Chemistry
    elif subcat <= 32:
        return '3' # for 4. Mechanical Engineering
    else:
        return '4' # for 5. Other fields

In [13]:
df_merge_quality['tech_field_big_cat'] = df_merge_quality['tech_field'].apply(tech_field_cat)

/home/nobu_yamaguchi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_merge_quality['breakthrough_flg'].value_counts()

0    371854
1       872
Name: breakthrough_flg, dtype: int64

In [14]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [15]:
MAX_SENT_LENGTH = 30
MAX_SENTS = 3
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [16]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [17]:
#df_margin_quality_filter = df_merge_quality[df_merge_quality['tech_field_big_cat'] == '1']
data_train = df_merge_quality.sample(frac=1)[:5000]
data_train

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6,breakthrough_flg
494607,EP-2639085-A1,EP-13154878-A,The method involves laminating a container (1)...,20130212,EP20130154878,380983662,EP20130154878,2013,32.0,1,...,1,NaN,NaN,0.687500,0.864198,0.500000,6.0,0.369032,0.383142,0
211476,EP-2614347-A1,EP-11761646-A,The invention relates in particular to a metho...,20110905,EP20110761646,337506971,EP20110761646,2011,10.0,1,...,0,NaN,NaN,NaN,0.750865,0.441176,7.0,0.497863,0.457056,0
492459,EP-2879255-A1,EP-13822860-A,An object of the present invention is to provi...,20130725,EP20130822860,414554662,EP20130822860,2013,2.0,1,...,0,NaN,NaN,NaN,0.815972,0.291667,5.0,0.088889,0.258975,0
358798,EP-2707705-A1,EP-12726900-A,To provide a surface property inspection devic...,20120509,EP20120726900,364495463,EP20120726900,2012,10.0,0,...,5,NaN,NaN,0.529422,0.617284,0.222222,6.0,0.654791,0.516557,0
242089,EP-2495573-A1,EP-11153731-A,A method and system are provided for obtaining...,20110208,EP20110153731,331620374,EP20110153731,2011,10.0,0,...,0,NaN,NaN,0.666667,0.777778,0.666667,8.0,0.429487,0.420065,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650102,EP-2997637-A1,EP-14724460-A,The invention relates to a method for adaptive...,20140516,EP20140724460,418261959,EP20140724460,2014,1.0,0,...,0,NaN,NaN,0.444444,0.444444,0.333333,4.0,0.476933,0.404528,0
811386,EP-3111207-A1,EP-15778196-A,The invention relates to a method of determini...,20150731,EP20150778196,445447034,EP20150778196,2015,10.0,1,...,0,NaN,NaN,NaN,0.795225,0.242424,3.0,0.257576,0.311349,0
90747,EP-2283795-A1,EP-10007783-A,"The supporting unit e.g. knee brace (10), has ...",20100727,EP20100007783,318014035,EP20100007783,2010,13.0,0,...,1,NaN,NaN,0.000000,0.460000,0.100000,8.0,0.192917,0.271628,0
31740,EP-2442922-A2,EP-10723104-A,The invention relates to a high-pressure clean...,20100602,EP20100723104,317432421,EP20100723104,2010,23.0,0,...,0,NaN,NaN,NaN,0.915713,0.870968,8.0,0.262339,0.260466,0


In [19]:
data_train.to_csv('patent_abstract_5000_2015.csv')

In [18]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence 
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data)
    labels.append(data_train.iloc[idx]['breakthrough_flg']) # label is just a list holding our label which is quality_index

In [19]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [217]:
tokenizer.word_counts

OrderedDict([('the', 51051),
             ('invention', 2808),
             ('is', 9909),
             ('based', 688),
             ('on', 3317),
             ('a', 37611),
             ('contactless', 20),
             ('current', 561),
             ('measuring', 266),
             ('device', 3224),
             ('10', 1653),
             ('for', 7335),
             ('determining', 366),
             ('consumption', 62),
             ('energy', 329),
             ('of', 22437),
             ('consumer', 19),
             ('14', 732),
             ('which', 3922),
             ('can', 1408),
             ('be', 2334),
             ('connected', 1023),
             ('by', 3761),
             ('means', 1293),
             ('plug', 120),
             ('32', 297),
             ('to', 15865),
             ('an', 8200),
             ('electrical', 447),
             ('ac', 70),
             ('or', 4646),
             ('three', 187),
             ('phase', 362),
             ('power', 949),
 

In [20]:
tokenizer.document_count

5000

In [21]:
len(tokenizer.word_index) # 143937 unqiue vocabulary out of the entire abstract dataset

17668

In [22]:
for i, sentences in enumerate(abstracts):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
            #for word in wordTokens:
                if word in tokenizer.word_index.keys():
                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

In [23]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 17668 unique tokens.


In [24]:
np.unique(np.asarray(labels))

array(['0', '1'], dtype='<U1')

In [25]:
labels = to_categorical(np.asarray(labels), num_classes=2) 
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 3, 30)
Shape of label tensor: (5000, 2)


In [26]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [27]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[3988.   12.]
[998.   2.]


In [28]:
x_train[2]

array([[    8,  7528,  1516,   129,    17,     7,    77,     5,    27,
          358,     5,     2,  1226,    66,    49,   259,     2,   362,
            3,   939,    17,   848,  1580,    21,     2,   114,    75,
           59,    17,     1],
       [    1,   939,    76,    27,    77,     5,  3398, 14475,    12,
         2347,  1101,     1,  1226,    66,     5,   391,  1433,     1,
         1226,    66,     5,     1,   129,     0,     0,     0,     0,
            0,     0,     0],
       [    1,  1226,    66,    76,    27,   310,    21,  1001,  1743,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]], dtype=int32)

In [29]:
x_val[10]

array([[ 319,    3,    1,   51,   25, 2977,    2,   28,    4,    8,   78,
           9,  483,    2, 9106,   58,   16,   53,    3,    2, 2753,  192,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   1,   28,   35, 1360,    2,  804, 2754,  351,  318, 9106,   58,
        1007,   16,    2,  920,   62,   64,    2,  423,   55,  437, 1966,
        4462,   55,   21,    1,   55,  437,   16,  604],
       [   1,   51,   25, 2884,    5,  141,    3,    2, 9106,   58,   16,
          53,    3,    2, 2753,  192,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [30]:
x_train.shape

(4000, 3, 30)

In [31]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops

import scipy.sparse


# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import glove_helper; reload(glove_helper)

<module 'glove_helper' from '/home/nobu_yamaguchi/final_project/glove_helper.py'>

In [32]:
GLOVE_DIR = "data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [33]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [34]:

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

(17669, 100)


In [38]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_sent)
l_lstm_sent_drop = Dropout(rate=0.5)(l_lstm_dense)
preds = Dense(2, activation='sigmoid')(l_lstm_sent_drop) # [Steven] why 5 will throw error and require 6 ? it is weird since we only have 5 number scale
model = Model(review_input, preds)

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [38]:
print("model fitting - Hierachical LSTM")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
3550/4000 [=========================>....] - ETA: 2s - loss: nan - acc: 0.5690

KeyboardInterrupt: 

In [39]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [40]:
print("model fitting - Hierachical LSTM")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=5, batch_size=50)

model fitting - Hierachical LSTM
Train on 4000 samples, validate on 1000 samples
Epoch 1/5
4000/4000 [==============================] - 11s 3ms/step - loss: 0.0591 - acc: 0.9893 - val_loss: 0.0147 - val_acc: 0.9980
Epoch 2/5
4000/4000 [==============================] - 9s 2ms/step - loss: 0.0273 - acc: 0.9970 - val_loss: 0.0144 - val_acc: 0.9980
Epoch 3/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.0233 - acc: 0.9970 - val_loss: 0.0144 - val_acc: 0.9980
Epoch 4/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.0238 - acc: 0.9970 - val_loss: 0.0172 - val_acc: 0.9980
Epoch 5/5
4000/4000 [==============================] - 8s 2ms/step - loss: 0.0217 - acc: 0.9970 - val_loss: 0.0146 - val_acc: 0.9980


In [42]:
pred = model.predict(x_val)
pred_output = []

for cnt in range(1000):
    if pred[cnt][0] > pred[cnt][1]: 
        pred_output.append(0)
    else:
        pred_output.append(1)

[i for i in pred_output if pred_output[i] != 0]

[]